In [1]:
import numpy as np
import pandas as pd
import spacy
from textblob import TextBlob

In [6]:
file_path = r"\Users\tailb\Downloads\amazon_product_reviews.csv"
amazon_data = pd.read_csv(file_path) # By default rows indexed.
#amazon_data = pd.read_csv(file_path,index_col=False) # index_col=False (rows indexed).
#amazon_data = pd.read_csv(file_path,index_col=0) # rows NOT indexed.

# Review first 5 rows.
amazon_data.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [8]:
# Recap Week - capstone.ipynb (preprocessing & post processing)
# Select the reviews columns.
amazon_data_to_clean = amazon_data[['reviews.text','reviews.title','reviews.username']]

# Review first 5 rows of the reviews columns.
amazon_data_to_clean.head()

,reviews.text,reviews.title,reviews.username
0,I thought it would be as big as small paper bu...,Too small,llyyue
1,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi
2,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo
3,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry
4,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack


 - Finding the sum of missing values in our Dataframe.

In [9]:
# isnull() method returns a DataFrame object where all the values are replaced with a Boolean value True for NULL values, and otherwise False.
# https://www.w3schools.com/python/pandas/ref_df_isnull.asp
# Detect missing values for an array-like object ( NaN in numeric arrays, None or NaN in object arrays, NaT in datetimelike).
amazon_data_to_clean.isnull().sum()

reviews.text         0
reviews.title       13
reviews.username     1
dtype: int64

 - Removing all rows with missing values in our Dataframe.

In [10]:
# dropna() - Remove all rows with NULL values.
# inplace = ** - If True: the removing is done on the current DataFrame. If False: returns a copy where the removing is done.
# axis = 0 or 'index' tells Pandas you want to remove rows. axis = 1 or 'columns tells Pandas you want to remove columns. 
# https://www.w3schools.com/python/pandas/ref_df_dropna.asp
amazon_data_to_clean.dropna(axis=0, inplace=True) 



C:\Users\tailb\AppData\Local\Temp\ipykernel_33072\3035700794.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_data_to_clean.dropna(axis=0, inplace=True)


- Review the sum of NULL values after cleaning with .dropna(axis=0, inplace=True).

In [11]:
# Review the sum of NULL values after cleaning with .dropna(axis=0, inplace=True).
amazon_data_to_clean.isnull().sum()

reviews.text        0
reviews.title       0
reviews.username    0
dtype: int64

 - Review columns where text preprocessing & postprocessing will be executed.

In [13]:
# Review columns where text preprocessing & postprocessing will be executed.
new_amazon_data = amazon_data_to_clean["reviews.text"]
new_amazon_data

0       I thought it would be as big as small paper bu...
1       This kindle is light and easy to use especiall...
2       Didnt know how much i'd use a kindle so went f...
3       I am 100 happy with my purchase. I caught it o...
4       Solid entry level Kindle. Great for kids. Gift...
                              ...                        
4995    This is a great tablet for the price. Amazon i...
4996    This tablet is the perfect size and so easy to...
4997    Purchased this for my son. Has room to upgrade...
4998    I had some thoughts about getting this for a 5...
4999    this is a steal, have 8 gb model as well.This ...
Name: reviews.text, Length: 4986, dtype: object

- Load advanced language model to find similarities and differences.
- Text preprocessing & postprocessing.


In [15]:
# Load advanced language model to find similarities and differences.
nlp = spacy.load('en_core_web_md')

# Define (function - any gived name)(value passed to the function - any given name):
def preprocessing(new_amazon_data):
    
    # Basic text cleaning.
    # .lower() method returns a string where all characters are lower case. Symbols and Numbers are ignored.
    # https://www.w3schools.com/python/ref_string_lower.asp
    # .strip() method removes any leading, and trailing whitespaces. Leading means at the beginning of the string, trailing means at the end. It ignores punctuations.:(
    # https://www.w3schools.com/python/ref_string_strip.asp
    doc = nlp(new_amazon_data.lower().strip())

    # Avoid returning tokens that are stop words and punctuations.
    # .lemma_ method (Lemmatisation) - Reduces a word to its base form, its 'mother word' to identify similarities.
    # https://builtin.com/machine-learning/lemmatization
    # .lower() method returns a string where all characters are lower case.
    postprocessing = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    
    # join() method takes all items in an iterable and joins them into one string. A string must be specified as the separator.
    # https://www.w3schools.com/python/ref_string_join.asp
    return ' '.join(postprocessing)

- Adding processed text to original dataframe without NULL values.
- Preparing the dataframe for Polarity & Sentiment Analysis.

In [16]:
# Funtion call.
# Adding processed text to original dataframe without NULL values.
amazon_data_to_clean["processed.text"] = amazon_data_to_clean["reviews.text"].apply(preprocessing)

# Review first 5 rows.
amazon_data_to_clean.head()

C:\Users\tailb\AppData\Local\Temp\ipykernel_33072\1685338508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_data_to_clean["processed.text"] = amazon_data_to_clean["reviews.text"].apply(preprocessing)


,reviews.text,reviews.title,reviews.username,processed.text
0,I thought it would be as big as small paper bu...,Too small,llyyue,think big small paper turn like palm think sma...
1,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,kindle light easy use especially beach
2,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,not know use kindle go low end m happy little ...
3,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,100 happy purchase catch sale good price norma...
4,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,solid entry level kindle great kid gift kid fr...


- Select product reviews using indexing.

In [17]:
# Select product reviews using indexing.
my_review_of_choice = amazon_data_to_clean["reviews.text"].apply(preprocessing)[2]
my_review_of_choice

'not know use kindle go low end m happy little dark'

- Use the .polarity attribute to measure the strength of sentiment in a product review.

In [18]:
# Open Session & Tutorial - NLP-polarity.ipynb (Visualising Polarity & Sentiment Analysis)
# Define (function - any gived name)(value passed to the function - any given name):
def analyze_polarity(my_review_of_choice):

    # Preprocess the text with spaCy
    doc = nlp(my_review_of_choice)
    
    # Analyze sentiment with TextBlob
    blob = TextBlob(my_review_of_choice)

    # Use .polarity to measure the strength of sentiment in a product review.
    # A polarity of 1 indicates a very positive sentiment.
    # A polaity of -1 indicates a very negative sentiment. 
    # A polarity of 0 indicates a neutral sentiment.
    polarity = blob.sentiment.polarity
    
    return polarity

# Function call
polarity_score = analyze_polarity(my_review_of_choice)

if polarity_score > 0:
    # If the first condition is evaluated to be True, the block of code below is executed.
    sentiment = 'positive sentiment'
elif polarity_score < 0:
    # If the first condition is evaluated to be FALSE, the block of code below is executed.
    sentiment = 'negative sentiment'
else:
     # If the first & second conditions are evaluated to be FALSE, the block of code below is executed.
    sentiment = 'neutral sentiment'

print(f"Text: {my_review_of_choice}\nPolarity score: {polarity_score:.2f}\nSentiment: {sentiment}")

Text: not know use kindle go low end m happy little dark
Polarity score: 0.12
Sentiment: positive sentiment


- Select product reviews using indexing.

In [19]:
# Select the first product review.
my_review_of_choice1 = amazon_data_to_clean["reviews.text"].apply(preprocessing)[0]
my_review_of_choice1

'think big small paper turn like palm think small read comfortable regular kindle definitely recommend paperwhite instead'

In [20]:
# Select the second product review.
my_review_of_choice2 = amazon_data_to_clean["reviews.text"].apply(preprocessing)[1]
my_review_of_choice2

'kindle light easy use especially beach'

In [21]:
# Select the 3rd product review.
my_review_of_choice3 = amazon_data_to_clean["reviews.text"].apply(preprocessing)[2]
my_review_of_choice3

'not know use kindle go low end m happy little dark'

- Using the .similarity() to compare the similarity of three product reviews.

In [22]:
nlp = spacy.load('en_core_web_md') # Load advanced language model to find similarities and differences.

review1 = nlp(my_review_of_choice1)
review2 = nlp(my_review_of_choice2) 
review3 = nlp(my_review_of_choice3)

print(review1.similarity(review2)) # The model has ascertained that review1 is 76% similar to review2.
print(review3.similarity(review2)) # The model has ascertained that review3 is 68% similar to review2.
print(review3.similarity(review1)) # The model has ascertained that review3 is 74% similar to review1.

0.7555949945656992
0.6813911169221817
0.7357000009652507
